In [9]:
rm -rf *tinydb jobs/*

In [10]:
import sys
import qgrid
import copy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sys.path.append('../../lib/')
from dftmanlib.pwscf import (pwinput_helper,
                             pwcalculation_helper,
                             pseudo_helper)
from dftmanlib.pwscf.workflow import EOSWorkflow
from dftmanlib.job import SubmitJob, submitjob_statuses, submit_status
from dftmanlib.matproj import mpquery_helper
from dftmanlib.db import init_db, load_db, MSONStorage

import tinydb
from tinydb.database import _get_doc_id

In [11]:
PSEUDO_TABLE = '/data/tools/shared/dftman/pseudo_table.json'
PSEUDO_FAMILY = 'GBRV_US_PBE'
MP_API_KEY = '0WqdPfXxloze6T9N'
qgrid.enable()

In [16]:
db = tinydb.TinyDB('db.tinydb', storage=MSONStorage)
table = db.table('_default')

In [17]:
criteria = {
    'elements': 'Al',
    'nsites': 1,
    'spacegroup.number': 225,
}
properties = []
m = mpquery_helper(criteria, properties, MP_API_KEY)
m.query()

STRUCTURE = m.result[0]['structure']
PSEUDO = pseudo_helper(STRUCTURE, PSEUDO_FAMILY,
                       PSEUDO_TABLE)
base_inputs = {
        'structure': STRUCTURE,

        'control': {
            'calculation': 'scf',
            'verbosity': 'high',
            'disk_io': 'none',
        },
        'system': {
            'ibrav': 0,
            'ecutwfc': None,
            'occupations': 'smearing',
            'degauss': 0.01,
            'smearing': 'mv',
        },
        'electrons': {
            'electron_maxstep': 500,
            'conv_thr': 1.0e-7,
        },
        'ions': {},
        'cell': {},
        'kpoints_mode': 'automatic',
        'kpoints_grid': (None, None, None),
        'kpoints_shift': (0, 0, 0),

        'pseudo': PSEUDO
    }
ecut = 35
# kpoints = [10, 12, 14, 16, 18, 20, 22, 24]
kpoints = [10]
kpoints_ids = []

for kpoint in kpoints:    
    inputs = copy.deepcopy(base_inputs)
    inputs['system']['ecutwfc'] = ecut
    inputs['kpoints_grid'] = (kpoint, kpoint, kpoint)
    
    runname = 'Alkpoint{}'.format(kpoint)
    calculation = pwcalculation_helper(**inputs, additional_inputs=list(PSEUDO.values()))
    job = SubmitJob(calculation, 'espresso-6.2.1_pw', runname=runname)
    
    doc_id = db.insert(job)

In [23]:
_get_doc_id(doc_id, None)
table._storage._storage.read()['_default']

{'1': {'@class': 'SubmitJob',
  '@module': 'dftmanlib.job.SubmitJob',
  'calculation': {'@class': 'PWCalculation',
                  '@module': 'dftmanlib.pwscf.pwscf',
                  'additional_inputs': ['/data/tools/shared/dftman/GBRV_PBE_US/al_pbe_v1.uspp.F.UPF'],
                  'directory': './',
                  'input_': {'@class': 'PWInput',
                             '@module': 'dftmanlib.pwscf.pwscf',
                             'cell': {},
                             'control': {'calculation': 'scf',
                                         'disk_io': 'none',
                                         'outdir': './',
                                         'prefix': 'pwscf',
                                         'pseudo_dir': './',
                                         'restart_mode': 'from_scratch',
                                         'verbosity': 'high'},
                             'electrons': {'conv_thr': 1e-07,
                                    